# DATA CLEANING

In [1]:
import pandas as pd
#df = pd.read_csv("data_scraped.csv")###############################################################################

df = pd.read_csv("glassdoor_jobs - Ken Jee - 2020.csv")


## Basic cleanings:
	* Drop duplicates and fill NaN values.
	* Remove first column (false index) and rows without salary.
	* Reset Index

In [2]:
#Checking columns names:
#df.columns
#dropping duplicates rows
df=df.drop_duplicates()
#deleting the undesirable column 1:
df = df.drop(['Unnamed: 0'], axis =1)
#deleting row without salary: our entirely analysis depends on salary info.
df = df[df['Salary Estimate'] != '-1']
#resetting index
df=df.reset_index(drop=True)
#df.isnull().sum() #This line count how many nan data are in each column of the df
#filling nan data with '-1'
df = df.fillna('-1')

## Parsed numeric data out of salary column as min, max and avg _salary:
	* Taked into account the hourly given salaries.
	* Added columns for employer provided salary and hourly wages.

In [3]:
#Since cells were found to have a different salary format, they should be highlighted.
#So two new columns with 1 for the detected cells and 0 for the normal cells are needed (one for each case):
df["hourly"] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df["employer_provided"] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

#Creating a new dataframe with the salary data and processed it:
    #Here we are deleting those rows without the salary data.
df = df[df['Salary Estimate'] != '-1']
    #Here is taken everything before the '(' ignoring the rest and pasting that in the serie "salary".
salary = df["Salary Estimate"].apply(lambda x: x.split('(')[0])
    #Here are erased the 'K' and '$
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))
    #Here are replaced the cases previously detected with a blank space instead.
min_hr = minus_Kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

#Now that only numbers are left, separated with a '-', split them and put them in two different dataframe will help to calculate the average
    #This doesn't take into consideration the data presented in an hourly form:
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))
    #calculating average:

#Here, Ken Jee forgot to take into consideration those salary values that where given in an hourly form. Maybe, I should had declared a serie and do it there. However, it works anyway:
    #It is logical to think tha the companies that are hiring telling an hourly salary could not be hiring annually, hiring by season, or are not sure about investment in this area, so  the salaries could be irregular (generally worst)
        #Later if I don't want to add them in the EDA step, then there we got the "hourly" column
            #Why I multiply by 1.92? 8 hours, 5 days, 4 weeks, 12 months: 8*5*4*12=1920... The values in the column are in miles, so I need to divide by 1000.
df['min_salary']= df.apply(lambda x: x['min_salary'] if x['hourly']==0 else (int)(x['min_salary']*1.92), axis = 1)
df['max_salary']= df.apply(lambda x: x['max_salary'] if x['hourly']==0 else (int)(x['max_salary']*1.92), axis = 1)
        
    #Finally, averaging column is calculated:
df['avg_salary'] = (df.min_salary+df.max_salary)/2

 ## Parsed rating out of company text and removed undesired characters.

In [4]:
#Taking into account that the Company_Name  column has not only the name of the company, but also the information of the ratings, it is necessary to remove ir the latter.
#If you know the glassdoor_scraping.py code you will know that when there is no rating, the cell will be filled with "-1", but when there is rating the content of the cell will be cero above (this also can be seen just looking the data).
#Finally, the rating (when exist) has 3 characters (number dot number)
#So, the next line is telling that when the Rating column has a value greater than zero, the company name will have 3 characters removed.
    #the "axis = 1" parameter says that the data will be searched in the same row
df['company_txt']= df.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'][:-3], axis = 1)

#If we watch what we have in the company names treated column:
df['company_txt']
#It is quite evident that each cell has "/n" characters at the end. Then we should remove it.

#Ken Jee used this:
#df['company_txt'] = df.company_txt.apply(lambda x: x.replace('\n',''))
#I think this is better:
df['company_txt'] = df['company_txt'].str[:-1]
df['company_txt']


0                          Tecolote Research
1      University of Maryland Medical System
2                                    KnowBe4
3                                       PNNL
4                         Affinity Solutions
                       ...                  
737                                      GSK
738                               Eventbrite
739           Software Engineering Institute
740                             Numeric, LLC
741             Riverside Research Institute
Name: company_txt, Length: 742, dtype: object

## Made a new column for company state and cleaned it.

In [5]:
#Here can be seeing that States are after the comma.
    #The next line split the cell and take what is after the comma. Simple.
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1] if ',' in x else x)
    #If we want to see how many are in each state (it prints the result in the console):
#df.job_state.value_counts()

#This works in Ken Jee work, but since the site glassdoor does not give HeadQuarters information anymore it is irrelevant to do this now. In any case, I left it here.
#df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis=1)


#Ok, now gonna see how the values of job_state are repeated
df['job_state'].value_counts()
#Here can be seen "Los Angeles" (an error, obviously)

df['job_state'] = df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'los angeles' else 'CA')
df['job_state'] = df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'iowa' else 'IA')
df['job_state'] = df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'georgia' else 'GA')
df['job_state'] = df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'virginia' else 'VA')
df['job_state'].value_counts()

## Transformed founded date into age of company 

In [9]:
#How many years are these companies working?
    #Here you must to put the year according to the data. If I use the data that Ken Jee extracted I must to use 2020 (in that year was the data extracted), then if I want to compare this data with data extracted other year and I don't use the correct year, the comparison become meaningless
df['age'] = df.Founded.apply(lambda x: x if x<1 else 2021-x)

## Made columns for if different skills were listed in the job description:

|||||||
|------------------|------------------|------------------|------------------|------------------|------------------|
|Python  |R|Spark|AWS|Excel|SQL|
|SAS|d3.js|Julia|Jupyter|Keras  |MatLab|
|MatPlotLib|PyTorch|Scikit-Learn|Tensor Flow|Weka|Selenium|
|Hadoop|Tableau|Power BI|BigML|RapidMiner|Apache Flink|
|DataRobot|SAP Hana|Mongo DB|Trifacta|MiniTab|Kafka|
|MicroStrategy|Google Analytics|SPSS|-|-|-|
||||||||

In [10]:
#We may see what we got.
#df['Job Description']
    #Looks like it is the full job description. Great! We can extract many information from this.

#Now, what Analysis should we do?
#Perhaps to watch how many job description mentions 2019 most used data science tool
    #Why 2019: cause Ken Jee is doing this analysis in 2020 and when he searched "top data science tool" to know (how one must do in every project "investigate what is important there") he found this site https://data-flair.training/blogs/data-science-tools/ and considered relevant enough.
    #This is just a little representation about how one must to investigate, take into consideration source with criterion and a judgment approach
    #So he determined (beginning with this example investigation) 5 keywords from the source: python, r studio, sparks, aws and excel.
    
    #Could this source be enough for me now (2021)?: https://searchbusinessanalytics.techtarget.com/feature/15-data-science-tools-to-consider-using 
                                                    #https://www.jigsawacademy.com/top-analytics-tools-every-data-scientist-must-learn/
    
    #Well, we count how many jobs_descriptions mention each keyword:
#python 
    #I think if some job description calls pySpark or py-something it is implicit "python"
df['python_yn']  = df['Job Description'].apply(lambda x: 1 if 'py' in x.lower() else 0)
    #NOTE: the _yn in the name means that what I'm doing is split by yes or no there is "python" in the description.
df.python_yn.value_counts()
#r studio
    #If you say "hey, I know r and py" looks like " r " is the correct way to search it.
df['R_yn']  = df['Job Description'].apply(lambda x: 1 if ' r ' in x.lower() or 'r-studio' in x.lower() or ' r,' in x.lower() or ' r.' in x.lower() else 0)
df.R_yn.value_counts()
#spark
df['spark_yn']  = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df.spark_yn.value_counts()
#aws
df['aws_yn']  = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df.aws_yn.value_counts()
#excel
df['excel_yn']  = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.excel_yn.value_counts()
#sql
df['sql_yn']  = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
df.sql_yn.value_counts()
#sas 
df['sas_yn']  = df['Job Description'].apply(lambda x: 1 if (' sas ' in x.lower() or ' sas,' in x.lower() or ' sas.' in x.lower())  else 0)
df.sas_yn.value_counts()
#d3js
df['d3js_yn']  = df['Job Description'].apply(lambda x: 1 if 'd3' in x.lower() else 0)
df.d3js_yn.value_counts()
#julia
df['julia_yn']  = df['Job Description'].apply(lambda x: 1 if 'julia' in x.lower() else 0)
df.julia_yn.value_counts()
#jupyter
df['jupyter_yn']  = df['Job Description'].apply(lambda x: 1 if 'jupyter' in x.lower() else 0)
df.jupyter_yn.value_counts()
#keras
df['keras_yn']  = df['Job Description'].apply(lambda x: 1 if 'keras' in x.lower() else 0)
df.keras_yn.value_counts()
#matlab
df['matlab_yn']  = df['Job Description'].apply(lambda x: 1 if 'matlab' in x.lower() else 0)
df.matlab_yn.value_counts()
#matplotlib
df['matplotlib_yn']  = df['Job Description'].apply(lambda x: 1 if 'matplotlib' in x.lower() else 0)
df.matplotlib_yn.value_counts()
#pytorch
df['pytorch_yn']  = df['Job Description'].apply(lambda x: 1 if 'pytorch' in x.lower() else 0)
df.pytorch_yn.value_counts()
#scikit-learn
df['scikit_yn']  = df['Job Description'].apply(lambda x: 1 if 'scikit' in x.lower() else 0)
df.scikit_yn.value_counts()
#tensor
df['tensor_yn']  = df['Job Description'].apply(lambda x: 1 if 'tensor' in x.lower() else 0)
df.tensor_yn.value_counts()
#weka
df['weka_yn']  = df['Job Description'].apply(lambda x: 1 if 'weka' in x.lower() else 0)
df.weka_yn.value_counts()
#selenium
df['selenium_yn']  = df['Job Description'].apply(lambda x: 1 if 'selenium' in x.lower() else 0)
df.selenium_yn.value_counts()
#hadoop
df['hadoop_yn']  = df['Job Description'].apply(lambda x: 1 if 'hadoop' in x.lower() else 0)
df.hadoop_yn.value_counts()
#tableau
df['tableau_yn']  = df['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df.tableau_yn.value_counts()
#power bi
df['bi_yn']  = df['Job Description'].apply(lambda x: 1 if ('power bi' in x.lower() or 'powerbi' in x.lower()) else 0)
df.bi_yn.value_counts()
#bigml
df['bigml_yn']  = df['Job Description'].apply(lambda x: 1 if 'bigml' in x.lower() else 0)
df.bigml_yn.value_counts()
#rapidminer
df['rapidminer_yn']  = df['Job Description'].apply(lambda x: 1 if 'rapidminer' in x.lower() else 0)
df.rapidminer_yn.value_counts()
#apache flink
df['flink_yn']  = df['Job Description'].apply(lambda x: 1 if 'flink' in x.lower() else 0)
df.flink_yn.value_counts()
#datarobot
df['datarobot_yn']  = df['Job Description'].apply(lambda x: 1 if 'datarobot' in x.lower() else 0)
df.datarobot_yn.value_counts()
#sap hana
df['hana_yn']  = df['Job Description'].apply(lambda x: 1 if 'hana' in x.lower() else 0)
df.hana_yn.value_counts()
#mongo db
df['mongo_yn']  = df['Job Description'].apply(lambda x: 1 if 'mongo' in x.lower() else 0)
df.mongo_yn.value_counts()
#trifacta
df['trifacta_yn']  = df['Job Description'].apply(lambda x: 1 if 'trifacta' in x.lower() else 0)
df.trifacta_yn.value_counts()
#minitab
df['minitab_yn']  = df['Job Description'].apply(lambda x: 1 if 'minitab' in x.lower() else 0)
df.minitab_yn.value_counts()
#kafka
df['kafka_yn']  = df['Job Description'].apply(lambda x: 1 if 'kafka' in x.lower() else 0)
df.kafka_yn.value_counts()
#microstrategy
df['microstrategy_yn']  = df['Job Description'].apply(lambda x: 1 if 'microstrategy' in x.lower() else 0)
df.microstrategy_yn.value_counts()
#google analytics
df['google_an_yn']  = df['Job Description'].apply(lambda x: 1 if 'google analytics' in x.lower() else 0)
df.google_an_yn.value_counts()
#Statistical Package for the Social Sciences
df['spss_yn']  = df['Job Description'].apply(lambda x: 1 if 'spss' in x.lower() else 0)
df.spss_yn.value_counts()

0    726
1     16
Name: spss_yn, dtype: int64

## Column for simplified Job Title.

In [11]:
#: there are words that are repeated more often than others.

#If we search for more extended types of data scientist, we will find just 5 or 6 (as Ken Jee done).
# def title_simplifier(title):
#     if 'data scientist' in title.lower():
#         return 'data scientist'
#     elif 'data engineer' in title.lower():
#         return 'data engineer'
#     elif 'analyst' in title.lower():
#         return 'analyst'
#     elif 'machine learning' in title.lower():
#         return 'machine learning'
#     elif 'manager' in title.lower():
#         return 'manager'
#     elif 'director' in title.lower():
#         return 'director'
#     else:
#         return 'na'
    
#but, If we read data we have got here we can find many more data related job positions. ALERT! doing this we can make a wrong 
#play, taking into account that we can be overfittint our model to the data we have, it could be better appreciated if we compare
#our results in this data with the results with the testing data or unseen data. If the results with other data is much more 
#different one of the reasons could be this overfitting
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data science' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'machine learning engineer'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    elif 'scientist' in title.lower():
        return 'other scientist'
    elif 'data modeler' in title.lower():
        return 'data modeler'
    elif 'data' and 'anal' in title.lower():
        return 'data analitics'
    else:
        return 'na'  #not applicable


In [12]:
df['job_simp'] = df['Job Title'].apply(title_simplifier)
df['job_simp'].value_counts()

data scientist               313
other scientist              143
data engineer                119
analyst                      101
machine learning engineer     22
manager                       16
na                            10
data analitics                 8
data modeler                   5
director                       5
Name: job_simp, dtype: int64

## Column for Seniority:
	* by analyzing Jobs Title info and by analyzing Jobs Descriptoin info.

In [13]:
#This function looks for the seniority asked in the title of each job asking (Job Title column)
def seniority(title):
    if 's.sr' in title.lower() or 's. sr' in title.lower() or 's sr' in title.lower() or 'ssr' in title.lower() or 's. senior' in title.lower() or 'semi or' in title.lower() or 'middle' in title.lower() or 'mid' in title.lower():
        return 'ssr'
    elif 'jr' in title.lower() or 'junior' in title.lower():
        return 'jr'
    elif 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'sr'
    else:
        return 'na'

na    519
sr    220
jr      3
Name: seniority_by_title, dtype: int64

In [ ]:
df['seniority_by_title'] = df['Job Title'].apply(seniority)
df.seniority_by_title.value_counts()

In [14]:
#This function looks for the seniority asked in the description of each job asking (Job Description column)
import re
        

def experience (job_description):

    if 'years experience' in job_description.lower(): #find each position in 'Job Description' that says "years experience" (years experience)
        text_list = job_description.split('years experience')
        text_list = [x[-15:] for x in text_list] #Just take last 15 characters before the searched string (where the text says how many years)
        
        if len(text_list)>1: #the next line only is executed if there is more than one item in the list
            text_list = text_list[:-1] #delete last item in list (cause it does not contain a number)
        
        # Here I transform words to numbers
        for index, item in enumerate(text_list):
            if "one" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('one','1')
            if "two" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('two','2')
            if "three" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('three','3')
            if "four" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('four','4')
            if "five" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('five','5')
            if "six" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('six','6')
            if "seven" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('seven','7')
            if "eight" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('eight','8')
            if "nine" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('nine','9')
            if "ten" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('ten','10')
            if "eleven" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('eleven','11')
            if "twelve" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('twelve','12')

        # Here numbers are everything left. (thanks stackoverflow)
        pattern = re.compile(r'\d+')
        num_list = [" ".join(pattern.findall(item)) for item in text_list] #Take each number and create a list of numbers.
        #NOTE: when there are decimals numbers, they are taken as different numbers (i.e. 3.5 are taken as 3 and 5). But, taking into account that the average between this and it correct value not affect much, it is irrelevant to correct it.

        num_list = max(num_list)   #finally we take the bigger number
        #(this means that if the company ask for 2 years of experience with python and 5 years with ML, the important number is the 5)
                
        text = ' '.join(num_list)
        #I choose to do it like this because if I join number I can't differentiate a decimal to decide which is larger('10 3' -> 103)
        last_list = text.split(' ')
        text = max(last_list)   #finally we take the bigger number
        #(this means that if the company ask for 2 years of experience with python and 5 years with ML, the important number is the 5)
             
        if text == '': #Writes 'na' when text is empty (because in the text before de searched string there is no numbers)
            text = 'na'
            
        return text
    
    elif 'years of' in job_description.lower(): #find each position in 'Job Description' that says "years of" (years of experience)
        text_list = job_description.split('years of')
        text_list = [x[-15:] for x in text_list] #Just take last 15 characters before the searched string (where the text says how many years)
        
        if len(text_list)>1: #the next line only is executed if there is more than one item in the list
            text_list = text_list[:-1] #delete last item in list (cause it does not contain a number)
        
        # Here I transform words to numbers
        for index, item in enumerate(text_list):
            if "one" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('one','1')
            if "two" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('two','2')
            if "three" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('three','3')
            if "four" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('four','4')
            if "five" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('five','5')
            if "six" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('six','6')
            if "seven" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('seven','7')
            if "eight" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('eight','8')
            if "nine" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('nine','9')
            if "ten" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('ten','10')
            if "eleven" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('eleven','11')
            if "twelve" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('twelve','12')

        # Here numbers are everything left. (thanks stackoverflow)
        pattern = re.compile(r'\d+')
        num_list = [" ".join(pattern.findall(item)) for item in text_list] #Take each number and create a list of numbers.
        #NOTE: when there are decimals numbers, they are taken as different numbers (i.e. 3.5 are taken as 3 and 5). But, taking into account that the average between this and it correct value not affect much, it is irrelevant to correct it.

        num_list = max(num_list)   #finally we take the bigger number
        #(this means that if the compay ask for 2 years of experience with python and 5 years with ML, the important number is the 5)
                
        text = ' '.join(num_list)
        #I choose to do it like this because if I join number I can't differentiate a decimal to decide which is larger('10 3' -> 103)
        last_list = text.split(' ')
        text = max(last_list)   #finally we take the bigger number
        #(this means that if the company ask for 2 years of experience with python and 5 years with ML, the important number is the 5)
    
        if text == '': #Writes 'na' when text is empty (because in the text before de searched string there is no numbers)
            text = 'na'    
    
        return text
    
    elif 'years’' in job_description.lower(): #find each position in 'Job Description' that says "years’" (year's experience)
        text_list = job_description.split('years’')
        text_list = [x[-15:] for x in text_list] #Just take last 15 characters before the searched string (where the text says how many years)
        
        if len(text_list)>1: #the next line only is executed if there is more than one item in the list
            text_list = text_list[:-1] #delete last item in list (cause it does not contain a number)
        
        # Here I transform words to numbers
        for index, item in enumerate(text_list):
            if "one" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('one','1')
            if "two" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('two','2')
            if "three" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('three','3')
            if "four" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('four','4')
            if "five" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('five','5')
            if "six" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('six','6')
            if "seven" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('seven','7')
            if "eight" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('eight','8')
            if "nine" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('nine','9')
            if "ten" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('ten','10')
            if "eleven" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('eleven','11')
            if "twelve" in text_list[index].lower():
                text_list[index] = text_list[index].lower().replace('twelve','12')

        # Here numbers are everything left. (thanks stackoverflow)
        pattern = re.compile(r'\d+')
        num_list = [" ".join(pattern.findall(item)) for item in text_list] #Take each number and create a list of numbers.
        #NOTE: when there are decimals numbers, they are taken as different numbers (i.e. 3.5 are taken as 3 and 5). But, taking into account that the average between this and it correct value not affect much, it is irrelevant to correct it.

        num_list = max(num_list)   #finally we take the bigger number
        #(this means that if the company ask for 2 years of experience with python and 5 years with ML, the important number is the 5)
                
        text = ' '.join(num_list)
        #I choose to do it like this because if I join number I can't differentiate a decimal to decide which is larger('10 3' -> 103)
        last_list = text.split(' ')
        text = max(last_list)   #finally we take the bigger number
        #(this means that if the company ask for 2 years of experience with python and 5 years with ML, the important number is the 5)

        if text == '': #Writes 'na' when text is empty (because in the text before de searched string there is no numbers)
            text = 'na'
        
        return text
   
    else:
        return 'na'
    
    
df['seniority_by_description'] = df['Job Description'].apply(experience)

df['seniority_by_description'].value_counts()


na    294
5     138
2     102
3      90
7      36
4      29
8      26
1      14
6      11
9       2
Name: seniority_by_description, dtype: int64

In [15]:
#Here I turn years of experience asked to seniority classification:
    #characterizing seniority by: https://devetry.com/blog/junior-vs-mid-vs-senior-software-engineers-–-experience-skills-expectations/
    
def years_2_seniority(years_c):
    if years_c.lower().isdigit():
        if int(years_c.lower()) <= 2:
            return 'jr'
        elif int(years_c.lower()) > 2 and int(years_c.lower()) <= 5:
            return 'ssr'
        elif int(years_c.lower()) > 5:
            return 'sr'
    else:
        return 'na'    
    
df['seniority_by_description'] = df['seniority_by_description'].apply(years_2_seniority)
df['seniority_by_description'].value_counts()

na     294
ssr    257
jr     116
sr      75
Name: seniority_by_description, dtype: int64

In [ ]:
df['seniority_by_description'] = df['seniority_by_description'].apply(years_2_seniority)
df['seniority_by_description'].value_counts()

In [16]:
#Finally I made de last seniorities characterization ranking by seniority_by_title and then seniority_by_description
df['seniority'] = df.apply(lambda x: x['seniority_by_description'] if 'na' in x['seniority_by_title'] else x['seniority_by_title'],axis=1)
df['seniority'].value_counts()

#job_title_variation=df.apply(lambda x: x['Job Title'] if x['job_simp']=="na" else 1,axis=1)

sr     258
na     214
ssr    179
jr      91
Name: seniority, dtype: int64

## Column for Job Description length
	* by quantity of letters and quantity of words

In [17]:
#It might be interesting to know if companies have longer descriptions if they are posting higher or lower salaries.
    #That is to say: Does a company want to explain away a low salary by having really in depth description???
    #later we gonna compare to answer this hypothesis, For now, just a columns with these values

#Counting characters:
df['desc_len'] = df['Job Description'].apply(lambda x:len(x))
df['desc_len']

0      2536
1      4783
2      3461
3      3883
4      2728
       ... 
737    6162
738    6130
739    3078
740    1642
741    3673
Name: desc_len, Length: 742, dtype: int64

In [18]:
#Counting words.
desc_wo_intro = df['Job Description']
#First I replace '\n' with ' ':
for index, item in enumerate(desc_wo_intro):
    if "\n" in desc_wo_intro[index].lower():
        desc_wo_intro[index] = desc_wo_intro[index].lower().replace('\n',' ')
# Then count words:
df['desc_len_words'] = desc_wo_intro.str.count(' ') + 1
df['desc_len_words']

C:\Users\s_eze\AppData\Local\Temp/ipykernel_8288/2538964308.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  desc_wo_intro[index] = desc_wo_intro[index].lower().replace('\n',' ')


0      356
1      681
2      494
3      532
4      378
      ... 
737    886
738    882
739    428
740    239
741    532
Name: desc_len_words, Length: 742, dtype: int64

## Checking the others columns:
	* Size
	* Type of ownership
	* Revenue
 

In [19]:
#Size Column
df['Size'] = df['Size'].apply(lambda x: 'na' if 'unknown' in x.lower() else ('na' if '-1' in x.lower() else x))
df['Size'].value_counts()

1001 to 5000 employees     150
501 to 1000 employees      134
10000+ employees           130
201 to 500 employees       117
51 to 200 employees         94
5001 to 10000 employees     76
1 to 50 employees           31
na                          10
Name: Size, dtype: int64

In [20]:
# Type of ownership 
df['Type of ownership'] = df['Type of ownership'].apply(lambda x: 'Other Organization' if 'unknown' in x.lower() else ('Other Organization' if '-1' in x.lower() else x))
df['Type of ownership'].value_counts()

Company - Private                 410
Company - Public                  193
Nonprofit Organization             55
Subsidiary or Business Segment     34
Government                         15
Hospital                           15
College / University               13
Other Organization                  5
School / School District            2
Name: Type of ownership, dtype: int64

In [21]:
#Revenue
df['Revenue'] = df['Revenue'].apply(lambda x: 'Unknown / Non-Applicable' if '-1' in x.lower() else x)
df['Revenue'].value_counts()

Unknown / Non-Applicable            204
$10+ billion (USD)                  124
$100 to $500 million (USD)           91
$1 to $2 billion (USD)               60
$500 million to $1 billion (USD)     57
$50 to $100 million (USD)            46
$25 to $50 million (USD)             40
$2 to $5 billion (USD)               39
$10 to $25 million (USD)             32
$5 to $10 billion (USD)              19
$5 to $10 million (USD)              18
$1 to $5 million (USD)                8
Less than $1 million (USD)            4
Name: Revenue, dtype: int64

## Finally exporting our data.

In [22]:
df_out=df
df_out.to_csv('____________data_cleaned.csv',index=False)

#Checking if the database was exported correctly:
#pd.read_csv('salary_data_cleaned.csv')
